In [1]:
%matplotlib inline
%pip install imagehash
%pip install tensorflow
%pip install -q transformers

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from transformers import PreTrainedTokenizerFast
from tokenizers import BertWordPieceTokenizer
from skimage.io import imread, imread_collection, imshow, imshow_collection
from scipy.signal import convolve2d
from scipy.fft import fft2,fftshift
from skimage.filters import threshold_otsu
from skimage.transform import resize, rescale, rotate
import os
# import tensorflow as tf
# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
# from tensorflow.keras.models import Model
# import ast
# from collections import Counter
# from PIL import Image
# import imagehash
#from itertools import combinations

C:\Users\nushi\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [4]:
diabets_set = pd.read_csv("original/diabetes.csv")
diabets_set

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,weight,frame,bp.1s,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn
0,1000,203.0,82,56.0,3.6,4.310000,Buckingham,46,female,62.0,121.0,medium,118.0,59.0,NaN,NaN,29.0,38.0,720.0
1,1001,165.0,97,24.0,6.9,4.440000,Buckingham,29,female,64.0,218.0,large,112.0,68.0,NaN,NaN,46.0,48.0,360.0
2,1002,228.0,92,37.0,6.2,4.640000,Buckingham,58,female,61.0,256.0,large,190.0,92.0,185.0,92.0,49.0,57.0,180.0
3,1003,78.0,93,12.0,6.5,4.630000,Buckingham,67,male,67.0,119.0,large,110.0,50.0,NaN,NaN,33.0,38.0,480.0
4,1005,249.0,90,28.0,8.9,7.720000,Buckingham,64,male,68.0,183.0,medium,138.0,80.0,NaN,NaN,44.0,41.0,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,41506,296.0,369,46.0,6.4,16.110001,Louisa,53,male,69.0,173.0,medium,138.0,94.0,130.0,94.0,35.0,39.0,210.0
399,41507,284.0,89,54.0,5.3,4.390000,Louisa,51,female,63.0,154.0,medium,140.0,100.0,146.0,102.0,32.0,43.0,180.0
400,41510,194.0,269,38.0,5.1,13.630000,Louisa,29,female,69.0,167.0,small,120.0,70.0,NaN,NaN,33.0,40.0,20.0
401,41752,199.0,76,52.0,3.8,4.490000,Louisa,41,female,63.0,197.0,medium,120.0,78.0,NaN,NaN,41.0,48.0,255.0


In [6]:
pd.read_csv("original/Diabetes Registry.CSV - Introduction to Biomedical Data Science.csv")

,id,chol,stab.glu,hdl,ratio,glyhb,Location,Age,Gender,Height,Weight 1,Weight 2,frame,bp.1s,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn
0,1000,203.0,82,56.0,3.6,4.310000,Buckingham,46,female,62.0,121,119,medium,118.0,59.0,NaN,NaN,29.0,38.0,720.0
1,1001,165.0,97,24.0,6.9,4.440000,Buckingham,29,female,64.0,218,218,large,112.0,68.0,NaN,NaN,46.0,48.0,360.0
2,1002,228.0,92,37.0,6.2,4.640000,Buckingham,58,female,61.0,256,243,large,190.0,92.0,185.0,92.0,49.0,57.0,180.0
3,1003,78.0,93,12.0,6.5,4.630000,Buckingham,67,male,67.0,119,121,large,110.0,50.0,NaN,NaN,33.0,38.0,480.0
4,1005,249.0,90,28.0,8.9,7.720000,Buckingham,64,male,68.0,183,179,medium,138.0,80.0,NaN,NaN,44.0,41.0,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,41503,301.0,90,118.0,2.6,4.280000,Louisa,89,female,61.0,115,121,medium,218.0,90.0,238.0,90.0,31.0,41.0,210.0
386,41506,296.0,369,46.0,6.4,16.110001,Louisa,53,male,69.0,173,189,medium,138.0,94.0,130.0,94.0,35.0,39.0,210.0
387,41507,284.0,89,54.0,5.3,4.390000,Louisa,51,female,63.0,154,154,medium,140.0,100.0,146.0,102.0,32.0,43.0,180.0
388,41510,194.0,269,38.0,5.1,13.630000,Louisa,29,female,69.0,167,170,small,120.0,70.0,NaN,NaN,33.0,40.0,20.0


In [9]:
pd.read_excel("original/Diabetes_Classification.xlsx")

,Patient number,Cholesterol,Glucose,HDL Chol,Chol/HDL ratio,Age,Gender,Height,Weight,BMI,Systolic BP,Diastolic BP,waist,hip,Waist/hip ratio,Diabetes,Unnamed: 16,Unnamed: 17
0,1,193,77,49,3.9,19,female,61,119,22.5,118,70,32,38,0.84,No diabetes,6.0,6.0
1,2,146,79,41,3.6,19,female,60,135,26.4,108,58,33,40,0.83,No diabetes,NaN,NaN
2,3,217,75,54,4.0,20,female,67,187,29.3,110,72,40,45,0.89,No diabetes,NaN,NaN
3,4,226,97,70,3.2,20,female,64,114,19.6,122,64,31,39,0.79,No diabetes,NaN,NaN
4,5,164,91,67,2.4,20,female,70,141,20.2,122,86,32,39,0.82,No diabetes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,386,227,105,44,5.2,83,female,59,125,25.2,150,90,35,40,0.88,No diabetes,NaN,NaN
386,387,226,279,52,4.3,84,female,60,192,37.5,144,88,41,48,0.85,Diabetes,NaN,NaN
387,388,301,90,118,2.6,89,female,61,115,21.7,218,90,31,41,0.76,No diabetes,NaN,NaN
388,389,232,184,114,2.0,91,female,61,127,24.0,170,82,35,38,0.92,Diabetes,NaN,NaN


In [12]:
pd.read_csv("data/codebook.csv")

,column_attribute,description
0,Patient number,Identifies patients by number
1,Cholesterol,Total cholesterol
2,Glucose,Fasting blood sugar
3,HDL,HDL or good cholesterol
4,Chol/HDL,Ratio of total cholesterol to good cholesterol...
5,Age,All adult African Americans
6,Gender,"162 males, 228 females"
7,Height,In inches
8,Weight,In pounds (lbs)
9,BMI,703 x weight (lbs)/ [height(inches]2


In [13]:
pd.read_csv("data/dates_times.csv")

,patient_id,date_of_birth,admission_date,admission_age,admit_year,admit_month,admit_day,weekday,admit_date
0,1,1972-11-03,2018-01-01,45.160849,2018,1,1,2,2018-01-01
1,2,1965-08-20,2018-02-05,52.462697,2018,2,5,2,2018-02-05
2,3,1955-04-01,2018-02-06,62.852841,2018,2,6,3,2018-02-06
3,4,1936-12-03,2018-03-04,81.248460,2018,3,4,1,2018-03-04
4,5,1956-03-04,2018-03-20,62.042437,2018,3,20,3,2018-03-20
5,6,1970-10-17,2018-04-17,47.498973,2018,4,17,3,2018-04-17
6,7,1950-09-24,2018-03-20,67.485284,2018,3,20,3,2018-03-20
7,8,1966-02-24,2018-04-20,52.150582,2018,4,20,6,2018-04-20
8,9,1999-09-05,2018-02-13,18.442163,2018,2,13,3,2018-02-13
9,10,1959-06-07,2018-01-24,58.633812,2018,1,24,4,2018-01-24


In [14]:
pd.read_csv("data/diabetes_classification.csv")

,patient_number,cholesterol,glucose,hdl_chol,chol_hdl_ratio,age,gender,height,weight,bmi,systolic_bp,diastolic_bp,waist,hip,waist_hip_ratio,diabetes
0,1,193,77,49,3.9,19,female,61,119,22.5,118,70,32,38,0.84,No diabetes
1,2,146,79,41,3.6,19,female,60,135,26.4,108,58,33,40,0.83,No diabetes
2,3,217,75,54,4.0,20,female,67,187,29.3,110,72,40,45,0.89,No diabetes
3,4,226,97,70,3.2,20,female,64,114,19.6,122,64,31,39,0.79,No diabetes
4,5,164,91,67,2.4,20,female,70,141,20.2,122,86,32,39,0.82,No diabetes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,386,227,105,44,5.2,83,female,59,125,25.2,150,90,35,40,0.88,No diabetes
386,387,226,279,52,4.3,84,female,60,192,37.5,144,88,41,48,0.85,Diabetes
387,388,301,90,118,2.6,89,female,61,115,21.7,218,90,31,41,0.76,No diabetes
388,389,232,184,114,2.0,91,female,61,127,24.0,170,82,35,38,0.92,Diabetes


In [15]:
pd.read_csv("data/diabetes_registry_csv_introduction_to_biomedical_data_science.csv")

,id,chol,stab_glu,hdl,ratio,glyhb,location,age,gender,height,weight_1,weight_2,frame,bp_1s,bp_1d,bp_2s,bp_2d,waist,hip,time_ppn
0,1000,203.0,82,56.0,3.6,4.310000,Buckingham,46,female,62.0,121,119,medium,118.0,59.0,NaN,NaN,29.0,38.0,720.0
1,1001,165.0,97,24.0,6.9,4.440000,Buckingham,29,female,64.0,218,218,large,112.0,68.0,NaN,NaN,46.0,48.0,360.0
2,1002,228.0,92,37.0,6.2,4.640000,Buckingham,58,female,61.0,256,243,large,190.0,92.0,185.0,92.0,49.0,57.0,180.0
3,1003,78.0,93,12.0,6.5,4.630000,Buckingham,67,male,67.0,119,121,large,110.0,50.0,NaN,NaN,33.0,38.0,480.0
4,1005,249.0,90,28.0,8.9,7.720000,Buckingham,64,male,68.0,183,179,medium,138.0,80.0,NaN,NaN,44.0,41.0,300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,41503,301.0,90,118.0,2.6,4.280000,Louisa,89,female,61.0,115,121,medium,218.0,90.0,238.0,90.0,31.0,41.0,210.0
386,41506,296.0,369,46.0,6.4,16.110001,Louisa,53,male,69.0,173,189,medium,138.0,94.0,130.0,94.0,35.0,39.0,210.0
387,41507,284.0,89,54.0,5.3,4.390000,Louisa,51,female,63.0,154,154,medium,140.0,100.0,146.0,102.0,32.0,43.0,180.0
388,41510,194.0,269,38.0,5.1,13.630000,Louisa,29,female,69.0,167,170,small,120.0,70.0,NaN,NaN,33.0,40.0,20.0


In [16]:
pd.read_csv("data/functions.csv")

,patient_id,fasting_insulin,fasting_glucose,converted_glucose,homa_score,high_homa_scores,glycohemoglobin
0,1.0,6.8,110,6.111111111111111,1.846914,1.0,6.0
1,2.0,9.5,90,5,2.111111,1.0,7.5
2,3.0,8.2,85,4.722222222222222,1.720988,0.0,8.6
3,4.0,7.2,105,5.833333333333333,1.866667,1.0,9.0
4,5.0,8.0,95,5.277777777777778,1.876543,1.0,5.5
5,NaN,NaN,Table 2,NaN,NaN,NaN,NaN
6,NaN,NaN,Patient ID,Glycohemoglobin,NaN,NaN,NaN
7,NaN,NaN,1,6,NaN,NaN,NaN
8,NaN,NaN,2,7.5,NaN,NaN,NaN
9,NaN,NaN,3,8.6,NaN,NaN,NaN


In [17]:
pd.read_csv("data/pivot_table.csv")

,column_a,column_b,column_c,column_d,column_e,column_f,column_g
0,1,120,NaN,NaN,NaN,Row Labels,Average of Glucose
1,1,128,NaN,NaN,NaN,1,122.33333333333333
2,1,119,NaN,NaN,NaN,2,109
3,2,110,NaN,NaN,NaN,3,140.25
4,2,108,NaN,NaN,NaN,4,107.5
5,3,140,NaN,NaN,NaN,5,90
6,3,145,NaN,NaN,NaN,(blank),NaN
7,3,150,NaN,NaN,NaN,Grand Total,116.5
8,3,126,NaN,NaN,NaN,NaN,NaN
9,4,105,NaN,NaN,NaN,NaN,NaN


In [18]:
pd.read_csv("data/sort_filter.csv")

,patient_number,cholesterol,glucose,hdl_chol,chol_hdl_ratio,age,gender,height,weight,bmi,systolic_bp,diastolic_bp,waist,hip,waist_hip_ratio,diabetes
0,1,193,77,49,3.9,19,female,61,119,22.5,118,70,32,38,0.84,No diabetes
1,2,146,79,41,3.6,19,female,60,135,26.4,108,58,33,40,0.83,No diabetes
2,3,217,75,54,4.0,20,female,67,187,29.3,110,72,40,45,0.89,No diabetes
3,4,226,97,70,3.2,20,female,64,114,19.6,122,64,31,39,0.79,No diabetes
4,5,164,91,67,2.4,20,female,70,141,20.2,122,86,32,39,0.82,No diabetes
5,6,170,69,64,2.7,20,female,64,161,27.6,108,70,37,40,0.93,No diabetes
6,7,149,77,49,3.0,20,female,62,115,21.0,105,82,31,37,0.84,No diabetes
7,8,164,71,63,2.6,20,male,72,145,19.7,108,78,29,36,0.81,No diabetes
8,9,230,112,64,3.6,20,male,67,159,24.9,100,90,31,39,0.79,No diabetes
9,10,179,105,60,3.0,20,female,58,170,35.5,140,100,34,46,0.74,No diabetes


In [19]:
pd.read_csv("data/visualization.csv")

,decade,glucose,age
0,A,77,19
1,A,79,19
2,A,75,20
3,A,97,20
4,A,91,20
...,...,...,...
385,G,105,83
386,G,279,84
387,G,90,89
388,H,184,91
